In [1]:
from pylab import rcParams
rcParams['figure.figsize'] = 10, 5
import warnings
warnings.filterwarnings('ignore')
from sklearn.tree import DecisionTreeClassifier
import pandas as pd
from sklearn.model_selection import GridSearchCV, train_test_split
from sklearn.metrics import confusion_matrix, classification_report, average_precision_score, accuracy_score
import csv

In [2]:
def get_algo_path():
    import os
    cwd = os.getcwd()
    s='\\'
    if '\\' not in cwd:
        s='/'
    cwd= cwd.split(s)[4:-1]
    cwd.append('Not Normalized')
    cwd='/'.join(cwd)
    return cwd

def get_csv_path():
    import os
    cwd = os.getcwd()
    s="\\"
    if "\\" not in cwd:
        s='/'
    file= cwd.split(s)[:4]
    file.append('models_scores.csv')
    file= s.join(file)
    return file

def line_is_exist(file, row):
    logfile = open(file, 'r')
    loglist = logfile.readlines()
    logfile.close()
    for line in loglist:
        if ','.join(row) in line:
            return True
    return False

def write_new_score(file, line):
    if( not line_is_exist(file, line) ):
        with open(file, 'a') as f:
            writer = csv.writer(f)
            writer.writerow(line)
    else:
        print('line exsist already')
    

In [3]:
data= pd.read_csv('../data.csv')

In [4]:
##copying data
data1=data.copy()
### spliting data en X et Y
X= data1.drop('Loan Status', axis=1)
Y= data1['Loan Status']
### spliting the data to train and test
X_train , X_test , Y_train , Y_test = train_test_split(X, Y, test_size = 0.30, random_state = 123)
## dicision tree (CART)
dct = DecisionTreeClassifier()
params={'criterion' : ['gini', 'entropy'],
        'max_depth': [4, 5, 6, 7, 8, 9, 10, 11, 12, 13,14, 15, 16, 17, None]}

##accuracy
grid_search_acc= GridSearchCV(estimator=dct, param_grid= params, scoring='accuracy',cv=2, n_jobs=-1)
grid_search_acc= grid_search_acc.fit(X_train, Y_train)
y_predict= grid_search_acc.best_estimator_.predict(X_test)


In [5]:
## get avg precision & avg recall
report= classification_report(Y_test, y_predict, output_dict=True)
avg_list = report.pop("weighted avg")
avg_precision= round(avg_list['precision'], 3)
avg_recall= round(avg_list['recall'], 3)
accuraccy= round(accuracy_score(Y_test,y_predict), 3)
## csv row
csv_row = [get_algo_path(), 'Decision Tree', str(grid_search_acc.best_params_), str(accuraccy), str(avg_precision), str(avg_recall)]
## write file
csv_file= get_csv_path()
write_new_score(csv_file, csv_row)

In [6]:
## evaluating the model
print("----------")
print('best parameters: ', grid_search_acc.best_params_)
print("----------")
print(confusion_matrix(Y_test,y_predict))
print("----------")
print(classification_report(Y_test, y_predict))
print("----------")


----------
best parameters:  {'criterion': 'entropy', 'max_depth': 12}
----------
[[11578  4456]
 [  576 15377]]
----------
              precision    recall  f1-score   support

           0       0.95      0.72      0.82     16034
           1       0.78      0.96      0.86     15953

    accuracy                           0.84     31987
   macro avg       0.86      0.84      0.84     31987
weighted avg       0.86      0.84      0.84     31987

----------


In [7]:
grid_search_acc.cv_results_



{'mean_fit_time': array([0.29471207, 0.33959174, 0.41838181, 0.46525609, 0.45977092,
        0.54803395, 0.56199646, 0.60488319, 0.58194375, 0.65325236,
        0.67718995, 0.72954953, 0.69115222, 0.77891707, 0.83526635,
        0.41239595, 0.49268258, 0.5634923 , 0.63929033, 0.67220259,
        0.67669082, 0.76345861, 0.83726108, 0.8841362 , 0.91106367,
        0.96043599, 1.10703909, 1.05517876, 1.03922176, 0.97791755]),
 'std_fit_time': array([0.00049758, 0.00149632, 0.0044874 , 0.01346338, 0.00199461,
        0.02144313, 0.04637623, 0.02443361, 0.00448895, 0.01396346,
        0.00399005, 0.04837072, 0.00299251, 0.02094364, 0.02243996,
        0.0244348 , 0.02094424, 0.03191423, 0.0129652 , 0.01994681,
        0.01745319, 0.04039133, 0.03241313, 0.00149608, 0.01446176,
        0.00698602, 0.02892315, 0.04089057, 0.03291297, 0.05934298]),
 'mean_score_time': array([0.01695383, 0.01994658, 0.01845109, 0.0179522 , 0.01745272,
        0.01795173, 0.01894975, 0.02044487, 0.01845038, 0.01